### Connect to database

In [74]:
import mariadb # librairie pour se connecter à mariadb
import sys
import var # import variable configured into var.py


# Connect to MariaDB Platform
try:
    conn = mariadb.connect(
        user=var.user,
        password=var.password,
        host=var.host,
        port=3306,
    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

# Get Cursor
cur = conn.cursor()


### Retrieve databases on the server
❗❗<code>SHOW DATABASES</code> query requires specific right on mariadb server
If our user do not get those rights, please run the command with another user
Some databases are not supposed to be transfered into Elastic, for exemple : <code>{'information_schema','mysql','sys','performance_schema'}</code>


In [75]:
cur.execute("SHOW DATABASES;") #* require specific rights on mariadb server

result = cur.fetchall() # retrieve all the results

cur.close()
# Create a list of databases name on the server
dbs_list= []
for row in result:
    dbs_list.append(row[0]) 

#! TODO : complete unwanted_db with any database that should not be transfered into Elastic
unwanted_db = {'information_schema','mysql','sys','performance_schema'}

#* dbs_list contain all databases that will be taken into account for batch
dbs_list = [db for db in dbs_list if db not in unwanted_db]

In [76]:
display(dbs_list)

['meteo1', 'mouleconnected']

### Get tables inside Database
We create a dictionnary <code>dict_db</code> with database name as key and value being a list  of tables inside the database

In [77]:
# for each db we want to transfer, we store tables names in the dictionnary
dict_db = {}
for db in dbs_list:
    cur = conn.cursor()
    query = f"SHOW TABLES FROM {db}"
    cur.execute(query)
    result = cur.fetchall()
    result = [value[0] for value in result] # keep only the name of the table
    dict_db[db] = result
    cur.close()


In [78]:
display(dict_db)

{'meteo1': ['DATA',
  'LOGBOOK',
  'ane0',
  'bar0',
  'cpr0',
  'gps0',
  'gsm0',
  'ntp0',
  'rad0',
  'the0'],
 'mouleconnected': ['Temperature',
  'bac',
  'capteur',
  'data',
  'data2',
  'lumiere',
  'moule',
  'users']}

Convert dict of list into dict of dict with list for values extracted

In [79]:
dict_db_tables = {}
for key, values in dict_db.items():
    dict_db_tables[key] = {value: {} for value in values}
display(dict_db_tables)

{'meteo1': {'DATA': {},
  'LOGBOOK': {},
  'ane0': {},
  'bar0': {},
  'cpr0': {},
  'gps0': {},
  'gsm0': {},
  'ntp0': {},
  'rad0': {},
  'the0': {}},
 'mouleconnected': {'Temperature': {},
  'bac': {},
  'capteur': {},
  'data': {},
  'data2': {},
  'lumiere': {},
  'moule': {},
  'users': {}}}

### Retrieve columns in tables

In [80]:
for db in dict_db.keys():
    for table in dict_db[db]:
        cur = conn.cursor()
        query = f"SHOW COLUMNS FROM {db}.{table}"
        cur.execute(query)
        result = cur.fetchall()
        cols = [value[0] for value in result]
        i=0
        for i in range(len(cols)):
            dict_db_tables[db][table][cols[i]] = []

display(dict_db_tables)
        

{'meteo1': {'DATA': {'utctimestamp': [], 'download': []},
  'LOGBOOK': {'utctimestamp': [], 'event': []},
  'ane0': {'utctimestamp': [], 'windspeed': [], 'winddirection': []},
  'bar0': {'utctimestamp': [], 'atmosphericpressure': []},
  'cpr0': {'utctimestamp': [],
   'uptime': [],
   'diskspace': [],
   'ramusage': [],
   'runningprocesses': [],
   'cpuload': []},
  'gps0': {'utctimestamp': [],
   'latitude': [],
   'longitude': [],
   'satellites': [],
   'hdop': [],
   'altitude': []},
  'gsm0': {'utctimestamp': [], 'atstatus': [], 'smsstatus': []},
  'ntp0': {'utctimestamp': [], 'offset': []},
  'rad0': {'utctimestamp': [], 'parirradiance': []},
  'the0': {'utctimestamp': [], 'airtemperature': []}},
 'mouleconnected': {'Temperature': {'IdBac': [], 'IdTemperature': []},
  'bac': {'Description': [], 'IdBac': []},
  'capteur': {'UniteMesure': [],
   'IdMoule': [],
   'IdTemperature': [],
   'IdLumiere': [],
   'IdBac': [],
   'IdCapteur': []},
  'data': {'DateMesure': [],
   'IdCapteu

### Extract data from table and insert it in the right column

In [81]:
for db in dict_db.keys():
    for table in dict_db[db]:
        cur = conn.cursor()
        query = f"SELECT * FROM {db}.{table}"
        cur.execute(query)
        result = cur.fetchall()
        i = 0
        j = 0
        for col in dict_db_tables[db][table]:
            for i in range(len(result)):
                dict_db_tables[db][table][col].append(result[i][j])
            j+=1
            
